In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd

hotels = pd.read_csv('../input/hotels/hotels.csv')
hotels.head()

In [3]:
print(hotels.info())

In [4]:
hotels['days_since_review'] = hotels['days_since_review'].apply(lambda x: x.split()[0]).astype('int')

hotels['review_date'] = pd.to_datetime(hotels['review_date'])  
hotels['review_date_month'] = hotels['review_date'].dt.month.astype(int)

In [5]:
hotels['review_date'].min()

In [6]:
unique_dic = {}
def make_unique_dic(x):
    x = x[3:-3].split(" ', ' ")
    for elem in x:
        if elem in unique_dic:
            unique_dic[elem] = unique_dic[elem] + 1
        else:
            unique_dic[elem] = 1
    return x

hotels['tags'].apply(make_unique_dic)
unique_tags = pd.DataFrame(unique_dic, index=['count']).transpose()
display(unique_tags)

In [7]:
unique_list=unique_tags.nlargest(20,'count').index.values.tolist()
unique_list

In [8]:
for tag in unique_list:
    hotels[tag] = hotels['tags'].apply(lambda x: 1 if tag in x else 0)
hotels=hotels.drop(['tags'], axis=1)

In [9]:
hotels

In [62]:
# убираем признаки которые еще не успели обработать, 
# модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
object_columns = [s for s in hotels.columns if hotels[s].dtypes == 'object']
hotels.drop(object_columns, axis = 1, inplace=True)

# заполняем пропуски самым простым способом
hotels = hotels.fillna(0)

In [63]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х - данные с информацией об отелях, у - целевая переменная (рейтинги отелей)  
X = hotels.drop(['reviewer_score'], axis = 1)  
y = hotels['reviewer_score'] 

In [49]:
hotels.info()

In [64]:
# Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split  

In [65]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
# Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [66]:
# Импортируем необходимые библиотеки:  
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  
  
# Создаём модель  
regr = RandomForestRegressor(n_estimators=100)  
      
# Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)  
      
# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)  


In [67]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они отличаются  
# Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднюю абсолютную процентную ошибку предсказанных значений от фактических.  
print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

In [36]:
# непрерывные признаки
num_cols = ['total_number_of_reviews', 'review_total_negative_word_counts','additional_number_of_scoring','review_total_positive_word_counts','total_number_of_reviews_reviewer_has_given',]

# категориальные признаки
cat_cols = ['average_score', 'lat']

In [37]:
y=y.astype('int')

...
from sklearn.feature_selection import chi2 # хи-квадрат

imp_cat = pd.Series(chi2(X[cat_cols], y)[0], index=cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

In [38]:
from sklearn.feature_selection import f_classif # anova

imp_num = pd.Series(f_classif(X[num_cols], y)[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

In [56]:
worst=['lat','additional_number_of_scoring','total_number_of_reviews','total_number_of_reviews_reviewer_has_given']
hotels.drop(worst, axis = 1, inplace=True)

In [71]:

hotels.info()